# Summary

Thanks for Dave to share the great PMC vectorization file: http://evexdb.org/pmresources/vec-space-models/

The vectorization part is developed from the tutorials by Francois Chollet.

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

A smaller LSTM is used. 
Smaller sequence length.

In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from gensim.models import KeyedVectors
import pickle
from sklearn.model_selection import GridSearchCV, train_test_split
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.optimizers import Adam



MAX_SEQUENCE_LENGTH = 100
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 200

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1 Vectorization

In [2]:
embeddings_index = KeyedVectors.load_word2vec_format('PubMed-and-PMC-w2v.bin', binary=True)

In [3]:
p_file = 'data_X'

with open(p_file, 'rb') as fin:
    data_X = pickle.load(fin)

In [4]:
p_file = 'RCT_labels'

with open(p_file, 'rb') as fin:
    labels = pickle.load(fin)

In [5]:
# vectorize the text into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(data_X)
sequences = tokenizer.texts_to_sequences(data_X)

In [6]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 1805160 unique tokens.
Shape of data tensor: (19111, 100)
Shape of label tensor: (19111, 2)


In [7]:
p_file = 'Models/vectors_small1'

with open(p_file, 'wb') as fout:
    pickle.dump(data, fout)

In [8]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    try:
        embedding_vector = embeddings_index[word]
    except KeyError:
        continue
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

        
p_file = 'Models/embedding_matrix_small1'

with open(p_file, 'wb') as fout:
    pickle.dump(embedding_matrix, fout)
    


Preparing embedding matrix.


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25)

In [10]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(MAX_NUM_WORDS,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# 2 LSTM

In [11]:
def model():
        
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    
    # Cov layer
    X = Conv1D(filters=256, kernel_size=15, strides=4)(embedded_sequences)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.6)(X)                                 # dropout
    
    # LSTM layer 1
    X = LSTM(128, return_sequences=True, recurrent_dropout = 0.5)(X)
    X = Dropout(0.5)(X)
    
    # LSTM layer 2 
    X = LSTM(128, recurrent_dropout = 0.5)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)  
    # Softmax layer
    X = Activation('softmax')(X)
    
    # Create Model instance
    model = Model(inputs = sequence_input, outputs = X)
        
    return model

In [12]:
model = model()

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 200)          4000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 22, 256)           768256    
_________________________________________________________________
batch_normalization_1 (Batch (None, 22, 256)           1024      
_________________________________________________________________
activation_1 (Activation)    (None, 22, 256)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 22, 128)           197120    
__________

In [14]:
opt = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [15]:
# Fit the model
model.fit(X_train, Y_train, batch_size = 64, epochs=20)

Epoch 1/20
14333/14333 [==============================] - 31s 2ms/step - loss: 0.5016 - acc: 0.7654
Epoch 2/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.4499 - acc: 0.8002
Epoch 3/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.4348 - acc: 0.8096
Epoch 4/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.4256 - acc: 0.8148
Epoch 5/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.4163 - acc: 0.8206
Epoch 6/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.4078 - acc: 0.8243
Epoch 7/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.4053 - acc: 0.8264
Epoch 8/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.3988 - acc: 0.8293
Epoch 9/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.3943 - acc: 0.8282
Epoch 10/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.3896 - acc: 0.8314

In [16]:
model_json = model.to_json()
with open ("Models/LSTMMid.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("Models/LSTMMid.h5")

In [17]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

4778/4778 [==============================] - 5s 984us/step
Validation set accuracy =  0.8003348681706173
